## Sentiment analysis with imdb reviews

In this notebook we work with the IMDb dataset, it is a binary sentiment analysis dataset consisting of 50,000 reviews from the Internet Movie Database (IMDb) labeled as positive (1) or negative (0). The dataset contains an even number of positive and negative reviews. Only highly polarizing reviews are considered. A negative review has a score ≤ 4 out of 10, and a positive review has a score ≥ 7 out of 10. We will apply a very simple preprocessing to the textreviews and then train a baseline randomforest on bag of words features. We will compare the results of the bag of word randomforest with a neural network performace where we learn a  dense word-embedding for each word and then classify it to either positive (1) or negative (0). Finally we will use an inception-like architecture with 1D convolutions and globalpooling and see if we can improve the performace. You can test the trained network on new reviews from the internet or by writting your own review for a movie you like or don't like.

In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
%matplotlib inline

In [0]:
# Downloading the review and labels, if it does not exist
import urllib
import os
if not os.path.isfile('movie_data.csv'):
    urllib.request.urlretrieve(
    "https://www.dropbox.com/s/kvwi2nlrtk7axn9/movie_data.csv?dl=1",
    "movie_data.csv")

In [3]:
df = pd.read_csv('movie_data.csv', encoding='utf-8')
df[0:5]

,review,sentiment
0,I went and saw this movie last night after bei...,1
1,Actor turned director Bill Paxton follows up h...,1
2,As a recreational golfer with some knowledge o...,1
3,"I saw this film in a sneak preview, and it is ...",1
4,Bill Paxton has taken the true story of the 19...,1


shuffle the data and extract the first 15000 reviews with the sentiments


In [0]:
df=df.sample(frac=1,random_state=22).reset_index(drop=True)
X = df.loc[0:14999, 'review'].values
y = df.loc[0:14999, 'sentiment'].values

look at one review before we apply some text preprocessing

In [0]:
i=0
print(X[i])
print(y[i])

Here we do a very simple preprocessing, no stemming no lemmatization no stopwords removed  
codecredit: https://stackabuse.com/text-classification-with-python-and-scikit-learn/

In [0]:
documents = []

for i in range(0, len(X)):  
    # Remove all the special characters
    document = re.sub(r'\W', ' ', str(X[i]))
    # remove all single characters
    document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document)
    # Remove single characters from the start
    document = re.sub(r'\^[a-zA-Z]\s+', ' ', document) 
    # Substituting multiple spaces with single space
    document = re.sub(r'\s+', ' ', document, flags=re.I)
    # Removing prefixed 'b'
    document = re.sub(r'^b\s+', '', document)
    # Removing html stuff
    document = re.sub("br", '', document)
    # Substituting multiple spaces with single space
    document = re.sub(r'\s+', ' ', document, flags=re.I)
    # Converting to Lowercase
    document = document.lower()
    documents.append(document)

look at the review from before after the text preprocessing

In [0]:
documents[0]

## RF Baseline

Here we "vectorize" the bag of words. We choose to consider only the 6000 most frequent words and take only words that appear in at least 5 diffrent reviews, furthermore we ignore all words that appear in more than 70% of all reviews. 

In [0]:
from sklearn.feature_extraction.text import CountVectorizer  
vectorizer = CountVectorizer(max_features=6000, min_df=5, max_df=0.7)
 
X = vectorizer.fit_transform(documents).toarray()  

In [0]:
print(len(vectorizer.get_feature_names()))#length of all tokens
np.array(vectorizer.get_feature_names()[0:200])#the first 200 tokens

spliting the bag of words into a train valid and testset

In [0]:
X_train = X[0:5000]
y_train = y[0:5000]
X_val = X[5000:10000]
y_val = y[5000:10000]
X_test = X[10000:15000]
y_test = y[10000:15000]

the data more or less balanced

In [0]:
print(np.unique(y_train,return_counts=True))
print(np.unique(y_val,return_counts=True))
print(np.unique(y_test,return_counts=True))
print(X_train.shape)
print(X_val.shape)
print(X_test.shape)

bag of words for the first five reviews (very sparse representation), 5000 observations, with 6000 features


In [0]:
print(X_train[0:5])
print(X_train.shape)

let's train a radomforest on the bag of words features of the train set

In [0]:
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators=300, random_state=36)  
classifier.fit(X_train, y_train)

prediction on the test set with the accuracy and the confusion matrix

In [0]:
y_pred = classifier.predict(X_test)  
print(pd.crosstab(y_test, y_pred,rownames=['true'], colnames=['pred']))
print("Acc = ",np.sum(y_test==y_pred)/len(y_test))

## Neural network with wordembedding

In [0]:
X_train = documents[0:5000]
y_train = y[0:5000]
X_val = documents[5000:10000]
y_val = y[5000:10000]
X_test = documents[10000:15000]
y_test = y[10000:15000]

In [0]:
X_train[0]

In the next cell we tokenize all unique words in the reviews and transform them into a sequence of the corresponding integer number of the token that belongs to the word. For example we cound transform "the" into the number 7. Then we take the length of the longest review and we zeropad all other reviews to that length of the longest review, so all reviews have the same length. With the max_words parameter you can control how many words you want to use, in this case we choose the 6000 most frequent words.

In [0]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

max_words=6000

# fit tokenizer on all reviews
total_reviews = documents
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(total_reviews) 


# transform tokens to a sequence of integers
X_train_tokens =  tokenizer.texts_to_sequences(X_train)
X_val_tokens = tokenizer.texts_to_sequences(X_val)
X_test_tokens = tokenizer.texts_to_sequences(X_test)

# max length of all reviews
max_length = max([len(s.split()) for s in total_reviews])
print("longest review:",max_length,"words")

# define vocabulary size (unique words in all reviews)
vocab_size = len(tokenizer.word_index) + 1
print("number of unique words/tokens:",vocab_size)


# zeropad the sequences to have the "same" length 
X_train_pad = pad_sequences(X_train_tokens, maxlen=max_length, padding='post')
X_val_pad = pad_sequences(X_val_tokens, maxlen=max_length, padding='post')
X_test_pad = pad_sequences(X_test_tokens, maxlen=max_length, padding='post')

this is how our input for the neural network will look like, just a sequence of integer numbers

all unique tokens with the corresponding number

In [0]:
X_train_pad[0]

Lets look at the first 200 most frquent words.

In [0]:
np.array(list(tokenizer.word_index)[0:200])

definition of the network with an embedding layer in the input that maps the numbers (words) into vectors of the same size.
architecture inspired by: https://www.tensorflow.org/tutorials/keras/basic_text_classification


In [0]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding,GlobalAveragePooling1D,Dropout,GlobalMaxPool1D

EMBEDDING_DIM = 30

model = Sequential()
model.add(Embedding(max_words, EMBEDDING_DIM, input_length=(None)))
model.add(GlobalAveragePooling1D())
model.add(Dropout(0.5))
model.add(Dense(20, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.summary()

In [0]:
history=model.fit(X_train_pad, y_train, batch_size=64, epochs=40, validation_data=(X_val_pad, y_val), verbose=1)

In [0]:
# summarize history 
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'valid'], loc='lower right')
plt.show()
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'valid'], loc='upper right')
plt.show()

prediction on the test set with the accuracy and the confusion matrix. Are we better than the RF baseline?

In [0]:
y_pred = model.predict_classes(X_test_pad)  
y_pred=y_pred.reshape(len(y_pred))
print(pd.crosstab(y_test, y_pred,rownames=['true'], colnames=['pred']))
print("Acc = ",np.sum(y_test==y_pred)/len(y_test))


### We need to go deeper! 
Write your own review or test the model on some reviews of Inception or some other movie, note that we can use an arbitrary length for the review.  
Visit: https://www.imdb.com/title/tt1375666/reviews?ref_=tt_ov_rt  
remeber 0 is negative and 1 is postive

In [0]:
#paste new reviews here in the sample list
sample=list(["Inception is truly one of a kind. A concept which has long gestated in Christopher Nolan's mind, his eye for drama mixed with his large scale sensibilities ring true in Blockbuster season making Inception a true original in the sea of reboots, remakes and sequels.To try and explain Inceptions many plot twists and incredibly intelligent arcs, would be a foolish task. As Nolan himself has been reluctant to. The best way to approach the film would be with an open mind, if you are prepared to be taken on a ride of a lifetime, then trust that you 100% will. If Avatar was a seminal film in technology (although coming out as a rather poor film, in my opinion), then Inception is seminal in it's storytelling. With a 148 minute running time, you would expect a lot to take place, but what you wouldn't expect is the pace of it all. I did not think at one time in the film about how long was left. I was simply blown away by the depth in every single part of the film. If my enthusiasm for the storytelling aspect of the film has left you worried about the spectacle, then don't worry. They are, as hinted in the trailer, incredible, looking real and unbelievable simultaneously. The most pleasing thing about the action set pieces, is that they are genuinely used to illustrate the story, rather than to blow stuff up a la Michael Bay. With this complex movie in it's high concept, a stellar cast is needed. And Nolan as always, delivers with just that. This is vintage DiCaprio, perhaps only equalled in The Aviator, which is even more impressive as his role as Cobb in Inception is not a showy one, needing DiCaprio to be the constant at the centre of the film. And he pulls off Cobb's emotional contradictions sublimely. The rest of the cast members all shine in parts of the films, Cillian Murphy shows off his usually non-existent tender side, Gordon-Levitt bottles his usual charm for his confidently reserved turn as the reliable Arthur, Watanabe is devilish as the seemingly ambiguous Saito, Page shows why she's the next big female star and Tom Hardy revels in being the comic relief of the film compared to his recent turns as decidedly psychopathic characters. Overall, Nolan has indeed surpassed himself. He has created a world as expansive as his Gotham, a plot dwarfing the intricacies of Memento and one which blows The Prestige's cinematic reveal out of the water. This is truly unmissable cinema. Revel in it, we've still got to wait a whole two years before Batman 3.",
             "Based on reviews I was hoping this was a different American film in the sense that it will have substance, subtlety and that it will make me think. It did not, it did not and it did not again. It is your typical Hollywood flick with car chases, shooting galore, explosions, fistfights, pretty boys and girls - the whole nine yards to sell tickets to the ADHD generation of teenagers. I gave it 3 stars - instead of just 2 - because the special effects are absolutely astonishing. This film wishes to be clever but really is not and does not make a lot of sense either. It reminded me of the teachers/professors who were confusing on purpose in order to make us believe they are smarter than we were."])
sample_tokens=tokenizer.texts_to_sequences(sample)
#print(np.array(sample_tokens))
print(model.predict(np.array(sample_tokens[0:1])))
print(model.predict(np.array(sample_tokens[1:2])))

### Now it's your turn



*   Play around with the EMBEDDING_DIM, the max_words paramter and the network architecture.
*   How does the test performace change if you use globalmaxpooling after the embedding?
*   How does it affect the validation performace?
*   Think about if the sentences still make sense after the preprocessing.

### Optional: Neural network with wordembedding (inception like) with Keras functional api  

In [0]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Concatenate, Dropout, Embedding, Conv1D,GlobalMaxPooling1D,GlobalAveragePooling1D
EMBEDDING_DIM = 30


a = Input(shape=(max_length,))
x = Embedding(max_words, EMBEDDING_DIM)(a)
x1 = Conv1D(filters=50,kernel_size=(3),activation="relu",padding="same")(x)
x2 = Conv1D(filters=50,kernel_size=(5),activation="relu",padding="same")(x)
x3 = Conv1D(filters=50,kernel_size=(7),activation="relu",padding="same")(x)

g1 = GlobalAveragePooling1D()(x1)
g2 = GlobalAveragePooling1D()(x2)
g3 = GlobalAveragePooling1D()(x3)
conc= Concatenate()([g1,g2,g3])
conc = Dropout(0.5)(conc)
out= Dense(1, activation='sigmoid')(conc)
model = Model(inputs=a, outputs=out)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

In [0]:
history=model.fit(X_train_pad, y_train, batch_size=64, epochs=10, validation_data=(X_val_pad, y_val), verbose=1)

In [0]:
# summarize history 
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'valid'], loc='lower right')
plt.show()
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'valid'], loc='upper right')
plt.show()

In [0]:
y_pred = model.predict(X_test_pad)  
y_pred=np.reshape(np.round(y_pred,0),(len(y_pred)))
print(pd.crosstab(y_test, y_pred,rownames=['true'], colnames=['pred']))
print("Acc = ",np.sum(y_test==y_pred)/len(y_test))